# Running a SCF

In [ ]:
import Auto_baby

In [ ]:
print(Auto_baby)

In [ ]:
silicon_experiment = Auto_baby.Experiment()

In [ ]:
silicon_experiment.ensure_directories()

In [ ]:
silicon_experiment.ensure_required_files()

In [ ]:
silicon_experiment.scf_calculation()

In [ ]:
silicon_experiment.dos()

In [ ]:
silicon_experiment.band_structure()

# Running a SCF with Fireworks

## download MP files from their offical repo

## Convex_hull(optional)

In [ ]:
from mp_api.client import MPRester

with MPRester("5ypNyStQsaDUA2neNLD17P8v2KNB241U") as mpr:


## WorkFlow Setup

In [ ]:
from fireworks import Firework, LaunchPad, ScriptTask, FileTransferTask, PyTask, Workflow
from fireworks.core.rocket_launcher import rapidfire
import Auto_baby

# # set up the LaunchPad and reset it
launchpad = LaunchPad()
launchpad.reset('', require_password=False)


_submit_SCF_Task_ = PyTask(func='Auto_baby.run_scf_for_FIREWORKS')
_transfer_CONTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/CONTCAR', 
                                                       'dest': '/home/student/zylu/demo/2_dos/POSCAR'}], 'mode': 'copy'})
_transfer_POTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/POTCAR', 
                                                       'dest': '/home/student/zylu/demo/2_dos/POTCAR'}], 'mode': 'copy'})
_transfer_JOB_of_SCF_to_DOS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/job.sh', 
                                                  'dest': '/home/student/zylu/demo/2_dos/job.sh'}], 'mode': 'copy'})
_transfer_WAVECAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/WAVECAR', 
                                                       'dest': '/home/student/zylu/demo/2_dos/WAVECAR'}], 'mode': 'copy'})
 
_submit_DOS_Task_ = PyTask(func='Auto_baby.run_dos_for_FIREWORKS')

_transfer_CONTCAR_of_SCF_to_BS = FileTransferTask({'files': [{f'src': '/home/student/zylu/demo/1_scf/CONTCAR', 
                                                  'dest': '/home/student/zylu/demo/3_bs/POSCAR'}], 'mode': 'copy'})
_transfer_POTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/POTCAR', 
                                                  'dest': '/home/student/zylu/demo/3_bs/POTCAR'}], 'mode': 'copy'})
_transfer_JOB_of_SCF_to_BS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/job.sh', 
                                                  'dest': '/home/student/zylu/demo/3_bs/job.sh'}], 'mode': 'copy'})
_transfer_CHGCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': '/home/student/zylu/demo/1_scf/CHGCAR', 
                                                  'dest': '/home/student/zylu/demo/3_bs/CHGCAR'}], 'mode': 'copy'})
_submit_BS_Task_ = PyTask(func='Auto_baby.run_band_for_FIREWORKS')

scf_calculation = Firework(
    tasks=[_submit_SCF_Task_],
    name="SCF Calculation" 
)


DOS_calculation = Firework(
    tasks=[_transfer_CONTCAR_of_SCF_to_DOS, _transfer_POTCAR_of_SCF_to_DOS, _transfer_JOB_of_SCF_to_DOS, _transfer_WAVECAR_of_SCF_to_DOS,_submit_DOS_Task_],
    name="DOS Calculation",
    parents=[scf_calculation]
)


BAND_structure_calculation = Firework(
    tasks=[_transfer_CONTCAR_of_SCF_to_BS, _transfer_POTCAR_of_SCF_to_BS, _transfer_JOB_of_SCF_to_BS, _transfer_CHGCAR_of_SCF_to_BS,_submit_BS_Task_],
    name="Band Structure Calculation",
    parents=[scf_calculation]
)

workflow_order = [scf_calculation, DOS_calculation, BAND_structure_calculation]

wf = Workflow(workflow_order, {scf_calculation.fw_id: [DOS_calculation.fw_id, BAND_structure_calculation.fw_id]})
launchpad.add_wf(wf)



rapidfire(launchpad)

In [5]:
def Final_Calculation(path='/home/student/zylu/demo'):
    _submit_SCF_Task_ = PyTask(func='Auto_baby.run_scf_for_FIREWORKS')
    _transfer_CONTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{path}/1_scf/CONTCAR', 
                                                       'dest': f'{path}/2_dos/POSCAR'}], 'mode': 'copy'})
    _transfer_POTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{path}/1_scf/POTCAR', 
                                                       'dest': f'{path}/2_dos/POTCAR'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{path}/1_scf/job.sh', 
                                                  'dest': f'{path}/2_dos/job.sh'}], 'mode': 'copy'})
    _transfer_WAVECAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{path}/1_scf/WAVECAR', 
                                                       'dest': f'{path}/2_dos/WAVECAR'}], 'mode': 'copy'})
    
    _submit_DOS_Task_ = PyTask(func='Auto_baby.run_dos_for_FIREWORKS')

    _transfer_CONTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{path}/1_scf/CONTCAR', 
                                                    'dest': f'{path}/3_bs/POSCAR'}], 'mode': 'copy'})
    _transfer_POTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{path}/1_scf/POTCAR', 
                                                    'dest': f'{path}/3_bs/POTCAR'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{path}/1_scf/job.sh', 
                                                    'dest': f'{path}/3_bs/job.sh'}], 'mode': 'copy'})
    _transfer_CHGCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{path}/1_scf/CHGCAR', 
                                                    'dest': f'{path}/3_bs/CHGCAR'}], 'mode': 'copy'})
    
    _submit_BS_Task_ = PyTask(func='Auto_baby.run_band_for_FIREWORKS')

    scf_calculation = Firework(
        tasks=[_submit_SCF_Task_],
        name="SCF Calculation" 
    )

    DOS_calculation = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_DOS, _transfer_POTCAR_of_SCF_to_DOS, _transfer_JOB_of_SCF_to_DOS, _transfer_WAVECAR_of_SCF_to_DOS,_submit_DOS_Task_],
        name="DOS Calculation",
        parents=[scf_calculation]
    )

    BAND_structure_calculation = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_BS, _transfer_POTCAR_of_SCF_to_BS, _transfer_JOB_of_SCF_to_BS, _transfer_CHGCAR_of_SCF_to_BS,_submit_BS_Task_],
        name="Band Structure Calculation",
        parents=[scf_calculation]
    )

    workflow_order = [scf_calculation, DOS_calculation, BAND_structure_calculation]

    wf = Workflow(workflow_order, {scf_calculation.fw_id: [DOS_calculation.fw_id, BAND_structure_calculation.fw_id]})
    launchpad.add_wf(wf)

    rapidfire(launchpad)

In [7]:
Final_Calculation()

2025-11-11 21:27:56,704 INFO Added a workflow. id_map: {-15: 4, -14: 5, -13: 6}
2025-11-11 21:27:56,706 INFO Created new dir /home/student/zylu/fireworks/launcher_2025-11-11-12-27-56-706324
2025-11-11 21:27:56,706 INFO Launching Rocket
2025-11-11 21:27:56,713 INFO RUNNING fw_id: 6 in directory: /home/student/zylu/fireworks/launcher_2025-11-11-12-27-56-706324
2025-11-11 21:27:56,716 INFO Task started: PyTask.
All calculations are organized in directories under /home/student/zylu/demo based on their names.
All required calculation inputs and job scripts are present where needed.
Current SLURM job state: UNKNOWN
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: COMPLETED
Final SLURM job state: COMPLETED
2025-11-11 21:28:03,402 INFO Task completed: PyTask
2025-11-11 21:28:03,412 INFO Rocket finished
2025-11-11 21:28:03,414 INFO Created new dir /home/st